<a href="https://colab.research.google.com/github/gaaseche/Profile-/blob/main/Delta_Airline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Business context**:

The dataset supports Delta Airlines’ efforts to enhance customer loyalty and operational efficiency by analyzing frequent flier behaviors, upcoming flight demands, and short-term occupancy trends. Key goals include improving personalized customer experiences, optimizing resource allocation, and maximizing ancillary revenue through tailored add-ons. This data-driven approach aids in strategic planning for customer retention and revenue growth. (ChatGPT)

**Structure**:

The dataset comprises three sheets with various structures:

1. Frequent Flier Raw: Contains frequent flier information such as "Frequent Flier Number," "Join Date," "First Name," etc., with a few columns needing renaming and structural adjustments.

2. Flier Upcoming Flights Raw: Tracks upcoming flights for each flier, including "Join Date," "Frequent Flier Number," "Flight Date," and related travel details. This sheet shows potential format inconsistencies in columns like "Cost," and issues in time zones.

3. 3 Month Upcoming Flights Raw: Details flights within the next three months, with flight times and city information. Here, time formatting inconsistencies and minor data entry errors are evident.

**Approach:**

1. Detect Missing Data: Identify and handle any null values appropriately.
2. Correct Inconsistencies: Standardize formats, resolve data types, and correct structural inconsistencies.
3. Remove Duplicates: If any duplicate records exist, we will remove them.
4. Document All Changes: Create a detailed record of each data cleaning step.


# **Step 1: Setup and File Upload**

In [38]:
!pip install openpyxl
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()



Saving Delta Airlines Loyalty Customer Raw Data.xlsx to Delta Airlines Loyalty Customer Raw Data (2).xlsx


# **Step 2: Load the Excel File and Inspect the Structure**

Replace 'Your_File_Name.xlsx' with the actual file name shown after upload

In [82]:
file_path = 'Delta Airlines Loyalty Customer Raw Data.xlsx'
excel_data = pd.ExcelFile(file_path)


Check the available sheet names

In [83]:
sheet_names = excel_data.sheet_names
print(sheet_names)

['Frequent Flier Raw', 'Flier Upcoming Flights Raw', '3 Month Upcoming Flights Raw']


Load Each Sheet into a DataFra



In [84]:
# Load the first sheet with header set to the second row to skip any instructional row
flier_raw_df = excel_data.parse('Frequent Flier Raw', header=1)
upcoming_flights_df = excel_data.parse('Flier Upcoming Flights Raw')
three_month_flights_df = excel_data.parse('3 Month Upcoming Flights Raw')


# **Step 3: Initial Data Review**

Examine Each DataFrame’s Structure and First Few Rows

In [85]:
print(flier_raw_df.head())
print(upcoming_flights_df.head())
print(three_month_flights_df.head())


  Record Creation   Frequent Flier Number            Join Date First Name  \
0       2024-02-27           7.234618e+09  1988-07-08 00:00:00     debbie   
1       2021-03-07           7.234618e+09  1988-07-08 00:00:00     debbie   
2       2023-01-31           7.234618e+09  1988-07-08 00:00:00     debbie   
3       2021-08-21           5.511217e+08  2022-06-16 00:00:00       kyle   
4       2022-03-01           5.511217e+08           16-06-2022       KYLE   

  Last Name   Inquiry Type Most Recent Flight #  Lounge Used?  \
0    spears  Flight Status            G35206241           0.0   
1    Spears     New Flight            V81311927           1.0   
2    spears  Cancel Flight            H82295055           0.0   
3     Boone     New Flight            E39886834           1.0   
4     Boone     New Flight            R04390271           NaN   

  Departing Airport Arrival Airport  Planned Snack?  Additional Snack?  \
0               DFW             DEN             0.0                1.0  

Skip the Instructional Row During Import:

In [86]:
# Print headers for each sheet
print(flier_raw_df.columns)
print(upcoming_flights_df.columns)
print(three_month_flights_df.columns)

Index(['Record Creation ', 'Frequent Flier Number', 'Join Date', 'First Name',
       'Last Name', 'Inquiry Type', 'Most Recent Flight #', 'Lounge Used?',
       'Departing Airport', 'Arrival Airport', 'Planned Snack?',
       'Additional Snack?', '# of Included Checked Bags',
       'Total # of Checked Bags', 'Flight Delayed?'],
      dtype='object')
Index(['Join Date', 'Frequent Flier Number', 'First Name', 'Last Name',
       'Flight Number', 'Flight Date', 'Departing City', 'Arrival City',
       'Departure Time', 'Arrival Time', 'Departure Time Zone',
       'Arrival Time Zone', 'Cost', 'Car Rental Addon $', 'Hotel Addon $'],
      dtype='object')
Index(['Flight_ID', 'Flight Date', 'Departure City', 'Arrival City',
       'Departure Time', 'Arrival Time', 'Departure Time Zone',
       'Arrival Time Zone', 'Airline', 'Flight Number', 'Seat Class', 'Price',
       'Currency', 'Available Seats', 'Baggage Allowance', 'Flight Duration',
       'Layovers', 'Cancellation Policy', 'Bookin

In [87]:
# Print the number of rows for each sheet
print("Number of rows in 'Frequent Flier Raw':", flier_raw_df.shape[0])
print("Number of rows in 'Flier Upcoming Flights Raw':", upcoming_flights_df.shape[0])
print("Number of rows in '3 Month Upcoming Flights Raw':", three_month_flights_df.shape[0])

Number of rows in 'Frequent Flier Raw': 9672
Number of rows in 'Flier Upcoming Flights Raw': 15078
Number of rows in '3 Month Upcoming Flights Raw': 20000


In [88]:
# Inspect column names and data types, and count missing values for each DataFrame
print("Columns and Info for 'Frequent Flier Raw':")
print(flier_raw_df.info())
print(f"\nMissing Values:\n{flier_raw_df.isnull().sum()}")

print("\nColumns and Info for 'Flier Upcoming Flights Raw':")
print(upcoming_flights_df.info())
print(f"\nMissing Values:\n{upcoming_flights_df.isnull().sum()}")

print("\nColumns and Info for '3 Month Upcoming Flights Raw':")
print(three_month_flights_df.info())
print(f"\nMissing Values:\n{three_month_flights_df.isnull().sum()}")



Columns and Info for 'Frequent Flier Raw':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9672 entries, 0 to 9671
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Record Creation             9211 non-null   datetime64[ns]
 1   Frequent Flier Number       9140 non-null   float64       
 2   Join Date                   9137 non-null   object        
 3   First Name                  9151 non-null   object        
 4   Last Name                   9140 non-null   object        
 5   Inquiry Type                9151 non-null   object        
 6   Most Recent Flight #        9153 non-null   object        
 7   Lounge Used?                9148 non-null   float64       
 8   Departing Airport           9173 non-null   object        
 9   Arrival Airport             9172 non-null   object        
 10  Planned Snack?              9143 non-null   float64       
 11  Additional Sn

In [89]:
# Summary of inspection:
# 1. Frequent Flier Data (9,672 rows x 15 cols):
#    - ~500 missing values across most cols
#    - Mix of datetime, float64, object types
#    - Most cols 95% complete

# 2. Future Bookings (15,078 rows x 15 cols):
#    - 753 missing values in select cols
#    - Mostly object types, 1 int col
#    - Critical fields (Flight #, Departure) complete

# 3. Flight Schedule (20,000 rows x 24 cols):
#    - 1,000 missing in key fields
#    - Large gaps: Layovers (9,975), Entertainment (6,114)
#    - Core flight info (ID, Status, Aircraft) complete
#    - Mostly object types, with float64, int64

In [79]:
# Print column names to inspect for unexpected characters or spaces
print("Column names in 'Frequent Flier Raw':")
print(flier_raw_df.columns)


Column names in 'Frequent Flier Raw':
Index(['Record Creation ', 'Frequent Flier Number', 'Join Date', 'First Name',
       'Last Name', 'Inquiry Type', 'Most Recent Flight #', 'Lounge Used?',
       'Departing Airport', 'Arrival Airport', 'Planned Snack?',
       'Additional Snack?', '# of Included Checked Bags',
       'Total # of Checked Bags', 'Flight Delayed?'],
      dtype='object')


In [90]:
# Convert date columns to datetime
flier_raw_df['Record Creation '] = pd.to_datetime(flier_raw_df['Record Creation'], errors='coerce')
flier_raw_df['Join Date'] = pd.to_datetime(flier_raw_df['Join Date'], errors='coerce')

# Verify the conversion
print(flier_raw_df[['Record Creation', 'Join Date']].dtypes)


KeyError: 'Record Creation'

In [91]:
# Reload the first sheet with header=1 to skip the instructional row
flier_raw_df = pd.read_excel(file_path, sheet_name='Frequent Flier Raw', header=1)

# Print column names to confirm exact names
print("Columns in 'Frequent Flier Raw':")
print(flier_raw_df.columns)

Columns in 'Frequent Flier Raw':
Index(['Record Creation ', 'Frequent Flier Number', 'Join Date', 'First Name',
       'Last Name', 'Inquiry Type', 'Most Recent Flight #', 'Lounge Used?',
       'Departing Airport', 'Arrival Airport', 'Planned Snack?',
       'Additional Snack?', '# of Included Checked Bags',
       'Total # of Checked Bags', 'Flight Delayed?'],
      dtype='object')


In [92]:
# Display the first few rows to confirm loading
print(flier_raw_df.head())


  Record Creation   Frequent Flier Number            Join Date First Name  \
0       2024-02-27           7.234618e+09  1988-07-08 00:00:00     debbie   
1       2021-03-07           7.234618e+09  1988-07-08 00:00:00     debbie   
2       2023-01-31           7.234618e+09  1988-07-08 00:00:00     debbie   
3       2021-08-21           5.511217e+08  2022-06-16 00:00:00       kyle   
4       2022-03-01           5.511217e+08           16-06-2022       KYLE   

  Last Name   Inquiry Type Most Recent Flight #  Lounge Used?  \
0    spears  Flight Status            G35206241           0.0   
1    Spears     New Flight            V81311927           1.0   
2    spears  Cancel Flight            H82295055           0.0   
3     Boone     New Flight            E39886834           1.0   
4     Boone     New Flight            R04390271           NaN   

  Departing Airport Arrival Airport  Planned Snack?  Additional Snack?  \
0               DFW             DEN             0.0                1.0  

In [93]:
# Convert 'Record Creation' and 'Join Date' to datetime
flier_raw_df['Record Creation'] = pd.to_datetime(flier_raw_df['Record Creation'], errors='coerce')
flier_raw_df['Join Date'] = pd.to_datetime(flier_raw_df['Join Date'], errors='coerce')

# Convert float columns with 1.0 and 0.0 to True/False
bool_columns = ['Lounge Used?', 'Planned Snack?', 'Additional Snack?', 'Flight Delayed?']
for col in bool_columns:
    flier_raw_df[col] = flier_raw_df[col].map({1.0: True, 0.0: False})

# Verify the changes
print(flier_raw_df[['Record Creation', 'Join Date']].head())
print(flier_raw_df[bool_columns].head())


KeyError: 'Record Creation'

In [94]:
# Strip any leading or trailing spaces from column names
flier_raw_df.columns = flier_raw_df.columns.str.strip()

# Verify the cleaned column names
print("Cleaned Column Names:", flier_raw_df.columns)

# Now, reattempt to convert 'Record Creation' and 'Join Date' to datetime
flier_raw_df['Record Creation'] = pd.to_datetime(flier_raw_df['Record Creation'], errors='coerce')
flier_raw_df['Join Date'] = pd.to_datetime(flier_raw_df['Join Date'], errors='coerce')

# Convert float columns with 1.0 and 0.0 to True/False
bool_columns = ['Lounge Used?', 'Planned Snack?', 'Additional Snack?', 'Flight Delayed?']
for col in bool_columns:
    flier_raw_df[col] = flier_raw_df[col].map({1.0: True, 0.0: False})

# Verify the changes
print(flier_raw_df[['Record Creation', 'Join Date']].head())
print(flier_raw_df[bool_columns].head())


Cleaned Column Names: Index(['Record Creation', 'Frequent Flier Number', 'Join Date', 'First Name',
       'Last Name', 'Inquiry Type', 'Most Recent Flight #', 'Lounge Used?',
       'Departing Airport', 'Arrival Airport', 'Planned Snack?',
       'Additional Snack?', '# of Included Checked Bags',
       'Total # of Checked Bags', 'Flight Delayed?'],
      dtype='object')
  Record Creation  Join Date
0      2024-02-27 1988-07-08
1      2021-03-07 1988-07-08
2      2023-01-31 1988-07-08
3      2021-08-21 2022-06-16
4      2022-03-01 2022-06-16
  Lounge Used? Planned Snack? Additional Snack? Flight Delayed?
0        False          False              True            True
1         True          False              True           False
2        False          False             False            True
3         True          False              True           False
4          NaN          False             False           False


In [95]:
# Fill missing numeric values with 0
numeric_cols = ['Frequent Flier Number', '# of Included Checked Bags', 'Total # of Checked Bags']
for col in numeric_cols:
    flier_raw_df[col] = flier_raw_df[col].fillna(0)

# Fill missing categorical values with 'Unknown'
categorical_cols = ['First Name', 'Last Name', 'Inquiry Type',
                    'Most Recent Flight #', 'Departing Airport', 'Arrival Airport']
for col in categorical_cols:
    flier_raw_df[col] = flier_raw_df[col].fillna('Unknown')

# Fill remaining NaN values in Boolean columns with False
for col in bool_columns:
    flier_raw_df[col] = flier_raw_df[col].fillna(False)

# Verify the changes by checking for any remaining missing values
print("Missing Values After Filling:")
print(flier_raw_df.isnull().sum())


Missing Values After Filling:
Record Creation               461
Frequent Flier Number           0
Join Date                     535
First Name                      0
Last Name                       0
Inquiry Type                    0
Most Recent Flight #            0
Lounge Used?                    0
Departing Airport               0
Arrival Airport                 0
Planned Snack?                  0
Additional Snack?               0
# of Included Checked Bags      0
Total # of Checked Bags         0
Flight Delayed?                 0
dtype: int64


<ipython-input-95-93bfbf15867f>:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  flier_raw_df[col] = flier_raw_df[col].fillna(False)


In [96]:
# Fill missing dates with pd.NaT (Not a Time) to indicate missing date data
flier_raw_df['Record Creation'] = flier_raw_df['Record Creation'].fillna(pd.NaT)
flier_raw_df['Join Date'] = flier_raw_df['Join Date'].fillna(pd.NaT)

# Verify if any missing values remain
print("Final Check for Missing Values:")
print(flier_raw_df.isnull().sum())


Final Check for Missing Values:
Record Creation               461
Frequent Flier Number           0
Join Date                     535
First Name                      0
Last Name                       0
Inquiry Type                    0
Most Recent Flight #            0
Lounge Used?                    0
Departing Airport               0
Arrival Airport                 0
Planned Snack?                  0
Additional Snack?               0
# of Included Checked Bags      0
Total # of Checked Bags         0
Flight Delayed?                 0
dtype: int64


In [97]:
# Drop rows where 'Record Creation' or 'Join Date' is missing
flier_raw_df.dropna(subset=['Record Creation', 'Join Date'], inplace=True)

# Verify that there are no remaining missing values
print("Check for Missing Values After Dropping Rows:")
print(flier_raw_df.isnull().sum())


Check for Missing Values After Dropping Rows:
Record Creation               0
Frequent Flier Number         0
Join Date                     0
First Name                    0
Last Name                     0
Inquiry Type                  0
Most Recent Flight #          0
Lounge Used?                  0
Departing Airport             0
Arrival Airport               0
Planned Snack?                0
Additional Snack?             0
# of Included Checked Bags    0
Total # of Checked Bags       0
Flight Delayed?               0
dtype: int64


In [ ]:
# Summary of Cleaning Steps for Frequent Flier Raw Converted Record Creation and Join Date to datetime.
# Converted Boolean-like columns to True and False.
# Filled missing values in numeric columns with 0.
# Filled missing categorical columns with "Unknown".
# Dropped rows with missing dates in Record Creation and Join Date (since they were critical).